In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MABarbadillo/
%cd 'GWP/GWP files'
import os

Mounted at /content/drive
/content/drive/MyDrive/MABarbadillo
/content/drive/MyDrive/MABarbadillo/GWP/GWP files


In [13]:
##Install rosetta

!pip install rosetta-soil
from rosetta import rosetta, SoilData

In [ ]:
import math
import numpy as np
import pandas as pd

# Read the CSV file, skipping the first row and using the second row as header
df = pd.read_csv('/content/drive/MyDrive/MABarbadillo/GWP/GWP files/MesoSoilv1_3.csv', header=0, skiprows=1)

# Extract the data from the specified columns
data = df.iloc[1:, [2, 3, 4, 5, 6, 7]].values.tolist()      #Columns 2,3,4,5,6, and 8 specify the columns for %sand, silt, clay, bulk density (g/cm3), and volumetric water contents (cm3/cm3) at -33 and -1500 kPa, respectively
K0 = df.iloc[1:, [13]].values.tolist()                      #K0 parameter from the csv file
L = df.iloc[1:, [14]].values.tolist()                       #L parameter from the csv file

# Call the function to get hydraulic parameters
mean_1, stdev_1, codes_1 = rosetta(1, SoilData.from_array(data))
mean_3, stdev_3, codes_3 = rosetta(3, SoilData.from_array(data))

# Extract the hydraulic parameter values from the array
theta_r = mean_1[0:, [0]].tolist()
theta_s = mean_1[0:, [1]].tolist()
alpha = mean_1[0:, [2]].tolist()
n = mean_1[0:, [3]].tolist()

# Calculate Saturated hydraulic conductivity
#m = 1 - (1 / n)
#theta = theta_r + (theta_s-theta_r) * (1 + (-alpha * 33)**n)**-m
#Se = (theta-theta_r)/(theta_s-theta_r)
#K_Se = (K0*Se**L) * (1 - (1-Se**(n/n-1))**m)**2


"""
Print results
print("Mean:")
print(mean_1)
print(mean_3)
print("Standard Deviation:")
print(stdev_3)
print(stdev_1)

"""